In [1]:
from datasets import Dataset, load_dataset

In [2]:
path = '/home/hv2237/GYAFC_Corpus/Entertainment_Music'
data = {}
for split in ['train']:
    data[split] = []
    for f, i in zip(open(f'{path}/{split}/formal').readlines(),open(f'{path}/{split}/informal').readlines()):
        data[split].append({'formal':f[:-1], 'informal':i[:-1]})
        
for split in ['tune', 'test']:
    data[split] = []
    refs = [open(f'{path}/{split}/formal.ref{i}').readlines() for i in range(4)]
    inp = open(f'{path}/{split}/informal').readlines()
    for f in range(len(inp)):
        temp = {}
        temp['informal'] = inp[f][:-1]
        for i in range(4):
            temp[f'formal.ref{i}'] = refs[i][f][:-1]
        data[split].append(temp)

In [3]:
data['test'][1]

{'informal': 'aspen colorado has he best music festivals, you sit all over the moutians its  on and just hang out',
 'formal.ref0': 'Aspen Colorado has he best music festivals, you sit all over the mountains it is on and just hang out.',
 'formal.ref1': "Aspen, CO has the best music festivals. You can sit on the mountains where they're held and simply enjoy.",
 'formal.ref2': 'Aspen Colorado is the home to some great music festivals, you can sit back enjoy the mountains and the music.',
 'formal.ref3': 'Aspen, Colorado has great music festivals. You sit over the mountains, and relax.'}

In [4]:
import json
for split in ['train', 'tune', 'test']:
    json.dump(data[split], open(f'{split}.json','w'))


In [5]:
train_dataset = load_dataset('json',data_files= {
    'train':'train.json'
})
eval_dataset = load_dataset('json',data_files={
    'valid':'tune.json',
    'test':'test.json'
})

Using custom data configuration default-1c5e66a391fdbc0e


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /home/hv2237/.cache/huggingface/datasets/json/default-1c5e66a391fdbc0e/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

...

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /home/hv2237/.cache/huggingface/datasets/json/default-3360e1aa3afb26f4/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-base')

2022-12-19 13:04:44.070118: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-19 13:04:44.612496: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-19 13:04:44.867580: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-19 13:04:46.610171: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [7]:
max_length = 30


def preprocess_function(examples):
    # print(examples)
    inputs = [ex for ex in examples["informal"]]
    targets = [ex for ex in examples["formal"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

def preprocess_function_eval(examples):
    # print(examples)
    inputs = [ex for ex in examples["informal"]]
    targets = [ex for ex in examples["formal.ref0"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    for i in range(4):
        model_inputs[f'formal{i}'] = examples[f'formal.ref{i}']
    return model_inputs



In [8]:
eval_dataset['test'][0]

{'informal': 'Is Any Baby Really A Freak.',
 'formal.ref0': 'Is any baby really a freak.',
 'formal.ref1': 'Is any baby really a freak?',
 'formal.ref2': 'Is Any Baby really strange?',
 'formal.ref3': 'Is anyone really that crazy?'}

In [9]:
tokenized_datasets = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset["train"].column_names,
)
tokenized_datasets_eval = eval_dataset.map(
    preprocess_function_eval,
    batched=True,
    remove_columns=eval_dataset["valid"].column_names,
)


  0%|          | 0/53 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [10]:
# tokenizer.decode(tokenized_datasets_eval['test'][0]['input_ids'], tokenized_datasets_eval['test'][0]

In [11]:
def collate(batch):
    keys = list(batch[0].keys())
    cols = {key:[] for key in keys}
    temp = list(map(lambda x: {'input_ids':x['input_ids'], 'labels':x['labels'], 'attention_mask':x['attention_mask']}, batch))
    for i in batch:
        for key in i:
            cols[key].append(i[key])
    out = data_collator(temp)
    for i in range(4):
        out[f'formal{i}'] = cols[f'formal{i}']
    return out

In [13]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained('bart-finetuned/best-model')

In [18]:
from huggingface_hub import notebook_login
notebook_login()

In [19]:
model.push_to_hub('formal-generation')

CommitInfo(commit_url='https://huggingface.co/talent404/formal-generation/commit/8394f2dbe937c18977d838447b742152c6e72ae1', commit_message='Upload BartForConditionalGeneration', commit_description='', oid='8394f2dbe937c18977d838447b742152c6e72ae1', pr_url=None, pr_revision=None, pr_num=None)

In [20]:
tokenizer.push_to_hub('formal-generation')

CommitInfo(commit_url='https://huggingface.co/talent404/formal-generation/commit/9b71e4bb1b4fee5bd74c40420c50dab0877f17aa', commit_message='Upload tokenizer', commit_description='', oid='9b71e4bb1b4fee5bd74c40420c50dab0877f17aa', pr_url=None, pr_revision=None, pr_num=None)

In [63]:
# from transformers import AutoModelForSeq2SeqLM
# model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-base')

In [14]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [15]:
from torch.utils.data import DataLoader

tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    collate_fn=data_collator,
    batch_size=32,
)
eval_dataloader = DataLoader(
    tokenized_datasets_eval["valid"],collate_fn=lambda x: collate(x) ,  batch_size=32
)

In [16]:
test_dataloader = DataLoader(
    tokenized_datasets_eval["test"],collate_fn=lambda x: collate(x) ,  batch_size=32
)

In [17]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
# device = "cpu"
model.to(device)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

In [68]:
from transformers import AdamW
import torch
# optimizer = AdamW(model.parameters(), lr=2e-5)
optimizer = torch.optim.Adam(model.parameters(),
                         betas=(0.9, 0.98), eps=1e-09, lr=3e-05, weight_decay=0.01)

In [69]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [70]:
# from transformers import get_scheduler
import transformers
num_train_epochs = 10
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

# lr_scheduler = get_scheduler(
#     "linear",
#     optimizer=optimizer,
#     num_warmup_steps=500,
#     num_training_steps=num_training_steps,
# )

lr_scheduler = transformers.get_polynomial_decay_schedule_with_warmup(optimizer, 500, 20000)


In [71]:
def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

In [22]:
1

1

In [72]:
import evaluate
bleurt = evaluate.load('bleurt', module_type="metric", checkpoint='bleurt-tiny-128')
def bleurt_loss(batch):
    labels = batch['labels']
    sampled_outputs = model.generate(
          input_ids=batch['input_ids'],
          attention_mask=batch['attention_mask'],
          output_scores=True,
          num_beams=1,
          do_sample=True,
          return_dict_in_generate=True,
          max_length=30,  
          top_k=tokenizer.vocab_size
        )
    greedy_outputs = model.generate(
          input_ids=batch['input_ids'],
          attention_mask=batch['attention_mask'],
          output_scores=True,
          do_sample=False,
          num_beams=1,
          max_length=30,
          return_dict_in_generate=True,
        )
    
    out = [i for i in sampled_outputs.scores[0]]
    for i in range(1,len(sampled_outputs.scores)):
        for j in range(len(out)):
            out[j] = torch.vstack((out[j],sampled_outputs.scores[i][j]))
    scores = torch.vstack(out).reshape(sampled_outputs.scores[0].shape[0], len(sampled_outputs.scores),-1)
    scores = torch.nn.functional.softmax(scores,dim=-1)
    # print(scores.shape)
    sample_idx, sampled_probs = torch.zeros(scores.shape[0], scores.shape[1]).to(accelerator.device), torch.zeros(scores.shape[0], scores.shape[1]).to(accelerator.device)
    for ind, i in enumerate(scores):
        temp = torch.multinomial(i, 1)
        sampled_probs[ind] = i.gather(1, temp.type(torch.int64)).squeeze(1)
        sample_idx[ind] = temp.squeeze(1)
    
    # print('Sampled Probs shape: ',sampled_probs.shape)
    sampled_texts = tokenizer.batch_decode(sampled_outputs.sequences, skip_special_tokens=True)
    greedy_texts = tokenizer.batch_decode(greedy_outputs.sequences, skip_special_tokens=True)
    labels = torch.where(labels != -100, labels, tokenizer.pad_token_id)
    label_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)


    # sample_bleu, greedy_bleu = [], []
    # i = 0
    # for ref, pred in zip(label_texts, sampled_texts):
    #     if len(pred)==0:
    #         # print(sampled_outputs.sequences[i], sampled_outputs.sequences.shape)
    #         # print(f"Ref: {ref}, Pred: {pred}")
    #         sample_bleu.append(0)
    #         continue
    #     sample_bleu.append(bleurt.compute(references= [ref], pred, smoothing_function=smooth.method1))
    #     i+=1
    # i = 0
    # for ref, pred in zip(label_texts, greedy_texts):
    #     if len(pred)==0:
    #         # print(greedy_outputs.sequences[i], greedy_outputs.sequences.shape)
    #         # print(f"Ref: {ref}, Pred: {pred}")
    #         greedy_bleu.append(0)
    #         continue
    #     greedy_bleu.append(sentence_bleu([ref], pred, smoothing_function=smooth.method1))
    #     i+=1
    sample_bleurt = bleurt.compute(references=label_texts, predictions=sampled_texts)['scores']
    greedy_bleurt = bleurt.compute(references=label_texts, predictions=greedy_texts)['scores']
    sample_bleurt = torch.FloatTensor(sample_bleurt).to(accelerator.device)
    greedy_bleurt = torch.FloatTensor(greedy_bleurt).to(accelerator.device)
    
    reward = (greedy_bleurt - sample_bleurt)*0.2
    
    # mask = batch['labels'].ne(tokenizer.pad_token_id)
    return rl_loss(sampled_probs, reward)

Using default BLEURT-Base checkpoint for sequence maximum length 128. You can use a bigger model for better results with e.g.: evaluate.load('bleurt', 'bleurt-large-512').


INFO:tensorflow:Reading checkpoint /home/hv2237/.cache/huggingface/metrics/bleurt/default/downloads/extracted/1f9e53fce958ea590db049fc3d3628f2bcffd5ff43cd4c69eab0d8a1e1123128/bleurt-base-128.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:128
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.


2022-12-17 04:37:58.745005: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-17 04:37:59.120772: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30324 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0


INFO:tensorflow:BLEURT initialized.


Taken from https://github.com/issacqzh/IRL_Table2Text/blob/main/Irl.py

In [24]:
import numpy as np

class Irl:

    def __init__(self,weights,size,learning_rate):
        # weights -> (1,reward_size)
        self.weights = weights
        self.normalized_weights = weights
        self.size = size
        self.learning_rate = learning_rate

    def update(self,cand_rewards,ref_rewards,cand_probs):
        cand_rewards = np.array(cand_rewards).transpose()
        ref_rewards = np.array(ref_rewards).transpose()
        # cand_rewards -> (reward_size,batch_size)
        cand_total_reward = np.dot(self.weights,cand_rewards)
    
        # cand_total_reward -> (batch_size)
        importance_weights = np.exp(cand_total_reward)/cand_probs
        check_inf = np.isinf(importance_weights)
        importance_weights[check_inf]=0

        relax_weights = [False]*len(self.weights)

        for i in range(len(self.weights)):
            ref_part = np.mean(ref_rewards[i])
            cand_part = np.sum(np.multiply(importance_weights,cand_rewards[i]))/np.sum(importance_weights)
            delta_weight = self.learning_rate*(ref_part-cand_part)
            if delta_weight < 0.00001:
                relax_weights[i] = True
            print(ref_part,cand_part,np.mean(cand_rewards[i]))
            self.weights[i] += delta_weight

        weights_sum = np.sum(self.weights)
        for i in range(len(self.weights)):
            self.normalized_weights[i] = self.weights[i]/weights_sum

        return relax_weights

In [25]:
def RL_all_rewards(batch):
    labels = batch['labels']
    sampled_outputs = model.generate(
          input_ids=batch['input_ids'],
          attention_mask=batch['attention_mask'],
          output_scores=True,
          num_beams=1,
          do_sample=True,
          return_dict_in_generate=True,
          max_length=30,  
          top_k=tokenizer.vocab_size
        )
    greedy_outputs = model.generate(
          input_ids=batch['input_ids'],
          attention_mask=batch['attention_mask'],
          output_scores=True,
          do_sample=False,
          num_beams=1,
          max_length=30,
          return_dict_in_generate=True,
        )
    
    out = [i for i in sampled_outputs.scores[0]]
    for i in range(1,len(sampled_outputs.scores)):
        for j in range(len(out)):
            out[j] = torch.vstack((out[j],sampled_outputs.scores[i][j]))
    scores = torch.vstack(out).reshape(sampled_outputs.scores[0].shape[0], len(sampled_outputs.scores),-1)
    scores = torch.nn.functional.softmax(scores,dim=-1)
    # print(scores.shape)
    sample_idx, sampled_probs = torch.zeros(scores.shape[0], scores.shape[1]).to(accelerator.device), torch.zeros(scores.shape[0], scores.shape[1]).to(accelerator.device)
    for ind, i in enumerate(scores):
        temp = torch.multinomial(i, 1)
        sampled_probs[ind] = i.gather(1, temp.type(torch.int64)).squeeze(1)
        sample_idx[ind] = temp.squeeze(1)
    
    # print('Sampled Probs shape: ',sampled_probs.shape)
    sampled_texts = tokenizer.batch_decode(sampled_outputs.sequences, skip_special_tokens=True)
    greedy_texts = tokenizer.batch_decode(greedy_outputs.sequences, skip_special_tokens=True)
    labels = torch.where(labels != -100, labels, tokenizer.pad_token_id)
    label_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    #Classifier Reward
    inps = clf_tok(sampled_texts , padding=True,truncation=True,max_length=30, return_tensors='pt').to(device)
    clf.eval()
    clf_out = clf(**inps)
    clf_probs = torch.nn.functional.softmax(clf_out['logits'], dim=-1)
    clf_reward =  clf_probs[:,1] - clf_probs[:,0]
    clf_ref_reward = 0
    if 'ref' in batch:
        inps = clf_tok(batch['ref'] , padding=True,truncation=True,max_length=30, return_tensors='pt').to(device)
        clf.eval()
        clf_out = clf(**inps)
        clf_probs = torch.nn.functional.softmax(clf_out['logits'], dim=-1)
        clf_ref_reward +=  clf_probs[:,1] - clf_probs[:,0]
    
    
    #BLEU Reward
    sample_bleu, greedy_bleu = [], []
    i = 0
    for ref, pred in zip(label_texts, sampled_texts):
        if len(pred)==0:
            # print(sampled_outputs.sequences[i], sampled_outputs.sequences.shape)
            # print(f"Ref: {ref}, Pred: {pred}")
            sample_bleu.append(0)
            continue
        sample_bleu.append(sentence_bleu([ref], pred, smoothing_function=smooth.method1))
        i+=1
    i = 0
    for ref, pred in zip(label_texts, greedy_texts):
        if len(pred)==0:
            # print(greedy_outputs.sequences[i], greedy_outputs.sequences.shape)
            # print(f"Ref: {ref}, Pred: {pred}")
            greedy_bleu.append(0)
            continue
        greedy_bleu.append(sentence_bleu([ref], pred, smoothing_function=smooth.method1))
        i+=1

    sample_bleu = torch.FloatTensor(sample_bleu).to(accelerator.device)
    greedy_bleu = torch.FloatTensor(greedy_bleu).to(accelerator.device)
    
    bleu_reward = (greedy_bleu - sample_bleu)
    bleu_ref_reward = (greedy_bleu - ref_bleu)
    #BLEURT Reward
    sample_bleurt = bleurt.compute(references=label_texts, predictions=sampled_texts)['scores']
    greedy_bleurt = bleurt.compute(references=label_texts, predictions=greedy_texts)['scores']
    sample_bleurt = torch.FloatTensor(sample_bleurt).to(accelerator.device)
    greedy_bleurt = torch.FloatTensor(greedy_bleurt).to(accelerator.device)
    
    bleurt_reward = (greedy_bleurt - sample_bleurt)
    
    return [rl_loss(sampled_probs, clf_reward), rl_loss(sampled_probs, bleu_reward), rl_loss(sampled_probs, bleurt_reward)]
    

In [73]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer 
clf = AutoModelForSequenceClassification.from_pretrained('style-classifier/checkpoint-1000', num_labels=2)

clf_tok = AutoTokenizer.from_pretrained('distilbert-base-uncased')

def classifier_loss(batch):
    sampled_outputs = model.generate(
          input_ids=batch['input_ids'],
          attention_mask=batch['attention_mask'],
          output_scores=True,
          num_beams=1,
          do_sample=True,
          return_dict_in_generate=True,
          max_length=30,  
          top_k=tokenizer.vocab_size
        )
    out = [i for i in sampled_outputs.scores[0]]
    for i in range(1,len(sampled_outputs.scores)):
        for j in range(len(out)):
            out[j] = torch.vstack((out[j],sampled_outputs.scores[i][j]))
    scores = torch.vstack(out).reshape(sampled_outputs.scores[0].shape[0], len(sampled_outputs.scores),-1)
    scores = torch.nn.functional.softmax(scores,dim=-1)
    # print(scores.shape)
    sample_idx, sampled_probs = torch.zeros(scores.shape[0], scores.shape[1]).to(accelerator.device), torch.zeros(scores.shape[0], scores.shape[1]).to(accelerator.device)
    for ind, i in enumerate(scores):
        temp = torch.multinomial(i, 1)
        sampled_probs[ind] = i.gather(1, temp.type(torch.int64)).squeeze(1)
        sample_idx[ind] = temp.squeeze(1)
    sampled_texts = tokenizer.batch_decode(sampled_outputs.sequences, skip_special_tokens=True)
    # print(sampled_texts)
    inps = clf_tok(sampled_texts , padding=True,truncation=True,max_length=30, return_tensors='pt').to(device)
    # print(inps, inps['input_ids'].shape)
    clf.to(device)
    clf.eval()
    clf_out = clf(**inps)
    clf_probs = torch.nn.functional.softmax(clf_out['logits'], dim=-1)
    clf_reward =  clf_probs[:,1] - clf_probs[:,0]
    
    
    return rl_loss(sampled_probs, clf_reward)
    
    

In [27]:
from nltk.translate.bleu_score import SmoothingFunction
smooth = SmoothingFunction()
from nltk.translate.bleu_score import sentence_bleu
import nltk
nltk.download('punkt')

def rl_loss(probs, reward):
    reward = reward.unsqueeze(1)
    sampled_logprobs = torch.log(probs)
    # print(sampled_logprobs.shape, mask.shape)
    # sampled_logprobs = torch.where(mask,sampled_logprobs, torch.Tensor(0.0))
    r_loss = -sampled_logprobs*reward
    # r_loss = r_loss.sum(-1)/mask.sum(-1)
    r_loss = r_loss.mean()
    return r_loss


def bleu_loss(batch):
    labels = batch['labels']
    sampled_outputs = model.generate(
          input_ids=batch['input_ids'],
          attention_mask=batch['attention_mask'],
          output_scores=True,
          num_beams=1,
          do_sample=True,
          return_dict_in_generate=True,
          max_length=30,  
          top_k=tokenizer.vocab_size
        )
    greedy_outputs = model.generate(
          input_ids=batch['input_ids'],
          attention_mask=batch['attention_mask'],
          output_scores=True,
          do_sample=False,
          num_beams=1,
          max_length=30,
          return_dict_in_generate=True,
        )
    
    out = [i for i in sampled_outputs.scores[0]]
    for i in range(1,len(sampled_outputs.scores)):
        for j in range(len(out)):
            out[j] = torch.vstack((out[j],sampled_outputs.scores[i][j]))
    scores = torch.vstack(out).reshape(sampled_outputs.scores[0].shape[0], len(sampled_outputs.scores),-1)
    scores = torch.nn.functional.softmax(scores,dim=-1)
    # print(scores.shape)
    sample_idx, sampled_probs = torch.zeros(scores.shape[0], scores.shape[1]).to(accelerator.device), torch.zeros(scores.shape[0], scores.shape[1]).to(accelerator.device)
    for ind, i in enumerate(scores):
        temp = torch.multinomial(i, 1)
        sampled_probs[ind] = i.gather(1, temp.type(torch.int64)).squeeze(1)
        sample_idx[ind] = temp.squeeze(1)
    
    # print('Sampled Probs shape: ',sampled_probs.shape)
    sampled_texts = tokenizer.batch_decode(sampled_outputs.sequences, skip_special_tokens=True)
    greedy_texts = tokenizer.batch_decode(greedy_outputs.sequences, skip_special_tokens=True)
    labels = torch.where(labels != -100, labels, tokenizer.pad_token_id)
    label_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)


    sample_bleu, greedy_bleu = [], []
    i = 0
    for ref, pred in zip(label_texts, sampled_texts):
        if len(pred)==0:
            # print(sampled_outputs.sequences[i], sampled_outputs.sequences.shape)
            # print(f"Ref: {ref}, Pred: {pred}")
            sample_bleu.append(0)
            continue
        sample_bleu.append(sentence_bleu([ref], pred, smoothing_function=smooth.method1))
        i+=1
    i = 0
    for ref, pred in zip(label_texts, greedy_texts):
        if len(pred)==0:
            # print(greedy_outputs.sequences[i], greedy_outputs.sequences.shape)
            # print(f"Ref: {ref}, Pred: {pred}")
            greedy_bleu.append(0)
            continue
        greedy_bleu.append(sentence_bleu([ref], pred, smoothing_function=smooth.method1))
        i+=1

    sample_bleu = torch.FloatTensor(sample_bleu).to(accelerator.device)
    greedy_bleu = torch.FloatTensor(greedy_bleu).to(accelerator.device)
    
    reward = (greedy_bleu - sample_bleu)*0.2
    
    # mask = batch['labels'].ne(tokenizer.pad_token_id)
    return rl_loss(sampled_probs, reward)
    

[nltk_data] Downloading package punkt to /home/hv2237/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [48]:
from tqdm.auto import tqdm
import torch
import numpy as np
import evaluate
metric_bleu = evaluate.load('bleu')
# metric_bleurt = evaluate.load('bleurt')

progress_bar = tqdm(range(num_training_steps))
steps = 0
best_bleu = 0.0
for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        
        
        
        if steps>200:
            if bleu:
                loss += bleu_loss(batch)
            if bleurt:
                loss += bleurt_loss(bacth)
            if classifier:
                loss += classifier_loss(batch)
        
        steps+=1
        
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
        if steps > 0 and steps%200 == 0:
            model.eval()
            predictions, references = [], []
            for batch in tqdm(eval_dataloader):
                with torch.no_grad():
                    generated_tokens = accelerator.unwrap_model(model).generate(
                        batch["input_ids"],
                        attention_mask=batch["attention_mask"],
                        max_length=128,
                    )
                labels = batch["labels"]

                # Necessary to pad predictions and labels for being gathered
                generated_tokens = accelerator.pad_across_processes(
                    generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
                )
                labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

                predictions_gathered = accelerator.gather(generated_tokens)
                labels_gathered = accelerator.gather(labels)

                decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
                predictions += decoded_preds

                transform = lambda x: [i for i in x]
                references += list(map(list, zip(*map(transform, [batch[f'formal{i}'] for i in range(4)]))))


            results_bleu = metric_bleu.compute(predictions=predictions, references=references)

            print(f"epoch {epoch}, BLEU score: {results_bleu['bleu']:.2f}")

            # Save and upload
            accelerator.wait_for_everyone()
            unwrapped_model = accelerator.unwrap_model(model)
            
            if results_bleu['bleu']>best_bleu:
                unwrapped_model.save_pretrained('bart-finetuned-fm/best-model', save_function=accelerator.save)
                best_bleu = results_bleu['bleu']
            unwrapped_model.save_pretrained('bart-finetuned-fm', save_function=accelerator.save)


  0%|          | 0/16240 [00:00<?, ?it/s]

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/88 [00:00<?, ?it/s]

epoch 0, BLEU score: 0.61


  0%|          | 0/88 [00:00<?, ?it/s]

...

  0%|          | 0/88 [00:00<?, ?it/s]

epoch 0, BLEU score: 0.73


  0%|          | 0/88 [00:00<?, ?it/s]

epoch 0, BLEU score: 0.72


KeyboardInterrupt: 

# IRL

In [ ]:
from tqdm.auto import tqdm
import torch
import numpy as np
import evaluate
metric_bleu = evaluate.load('bleu')
# metric_bleurt = evaluate.load('bleurt')

progress_bar = tqdm(range(num_training_steps))
steps = 0
best_bleu = 0.0
weights = [0]*5
irl_samples = 500
irl = Irl(weights, config.irl_size, config.irl_lr)

    # Training
model.train()

# Supervised learning
for batch in train_dataloader:
    outputs = model(**batch)
    loss = outputs.loss
    steps+=1

    accelerator.backward(loss)

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

# Extracting candidate and ref rewards
model.eval()
ref_rewards = []
cand_rewards = []
cand_probs = []

for batch in train_dataloader:

    labels = batch['labels']
    curr_samples += len(batch['labels'])
    sampled_outputs = model.generate(
          input_ids=batch['input_ids'],
          attention_mask=batch['attention_mask'],
          output_scores=True,
          num_beams=1,
          do_sample=True,
          return_dict_in_generate=True,
          max_length=30,  
          top_k=tokenizer.vocab_size
        )
    out = [i for i in sampled_outputs.scores[0]]
    for i in range(1,len(sampled_outputs.scores)):
        for j in range(len(out)):
            out[j] = torch.vstack((out[j],sampled_outputs.scores[i][j]))
    scores = torch.vstack(out).reshape(sampled_outputs.scores[0].shape[0], len(sampled_outputs.scores),-1)
    scores = torch.nn.functional.softmax(scores,dim=-1)
    # print(scores.shape)
    sample_idx, sampled_probs = torch.zeros(scores.shape[0], scores.shape[1]).to(accelerator.device), torch.zeros(scores.shape[0], scores.shape[1]).to(accelerator.device)
    for ind, i in enumerate(scores):
        temp = torch.multinomial(i, 1)
        sampled_probs[ind] = i.gather(1, temp.type(torch.int64)).squeeze(1)
        sample_idx[ind] = temp.squeeze(1)

    # print('Sampled Probs shape: ',sampled_probs.shape)
    sampled_texts = tokenizer.batch_decode(sampled_outputs.sequences, skip_special_tokens=True)
    labels = torch.where(labels != -100, labels, tokenizer.pad_token_id)
    label_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)

    #BLEU

    sampled_texts = tokenizer.batch_decode(sampled_outputs.sequences, skip_special_tokens=True)
    greedy_texts = tokenizer.batch_decode(greedy_outputs.sequences, skip_special_tokens=True)
    labels = torch.where(labels != -100, labels, tokenizer.pad_token_id)
    label_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)


    sample_bleu, greedy_bleu = [], []
    i = 0
    for ref, pred in zip(label_texts, sampled_texts):
        if len(pred)==0:
            # print(sampled_outputs.sequences[i], sampled_outputs.sequences.shape)
            # print(f"Ref: {ref}, Pred: {pred}")
            sample_bleu.append(0)
            continue
        sample_bleu.append(sentence_bleu([ref], pred, smoothing_function=smooth.method1))
        i+=1
    i = 0
    for ref, pred in zip(label_texts, greedy_texts):
        if len(pred)==0:
            # print(greedy_outputs.sequences[i], greedy_outputs.sequences.shape)
            # print(f"Ref: {ref}, Pred: {pred}")
            greedy_bleu.append(0)
            continue
        greedy_bleu.append(sentence_bleu([ref], pred, smoothing_function=smooth.method1))
        i+=1

    sample_bleu = torch.FloatTensor(sample_bleu).to(accelerator.device)
    greedy_bleu = torch.FloatTensor(greedy_bleu).to(accelerator.device)

    cand_reward = (greedy_bleu - sample_bleu)
    ref_reward = torch.ones_like(cand_reward)

    #BLEURT

    sample_bleurt = bleurt.compute(references=label_texts, predictions=sampled_texts)['scores']
    greedy_bleurt = bleurt.compute(references=label_texts, predictions=greedy_texts)['scores']
    sample_bleurt = torch.FloatTensor(sample_bleurt).to(accelerator.device)
    greedy_bleurt = torch.FloatTensor(greedy_bleurt).to(accelerator.device)

    ref_bleurt = bleurt.compute(references=label_texts, predictions=label_texts)['scores']
    ref_bleurt = torch.FloatTensor(ref_bleurt).to(accelerator.device)

    cand_bleurt_reward = (greedy_bleurt - sample_bleurt)*0.2


    #Classifier Reward

    inps = clf_tok(sampled_texts , padding=True,truncation=True,max_length=30, return_tensors='pt').to(device)
    clf.to(device)
    clf.eval()
    clf_out = clf(**inps)
    clf_probs = torch.nn.functional.softmax(clf_out['logits'], dim=-1)
    clf_reward =  clf_probs[:,1] - clf_probs[:,0]

    inps = clf_tok(label_texts , padding=True,truncation=True,max_length=30, return_tensors='pt').to(device)
    clf_out = clf(**inps)
    clf_probs = torch.nn.functional.softmax(clf_out['logits'], dim=-1)
    clf_ref_reward =  clf_probs[:,1] - clf_probs[:,0]

    cand_rewards = [cand_reward, cand_bleurt_reward, clf_reward]
    ref_rewards = [ref_reward, ref_bleurt, clf_ref_reward]
    cand_probs = sampled_probs
        
for epoch in irl_epochs:
    for _ in range(2):
        ref_rewards = np.array(ref_rewards)
        cand_rewards = np.array(cand_rewards)
        cand_probs = np.array(cand_probs)
        
        indice=random.sample(range(ref_rewards.shape[0]), irl_samples)

        sample_ref_rewards=ref_rewards[indice]
        sample_cand_rewards=cand_rewards[indice]
        sample_cand_probs=cand_probs[indice]
        
        irl.update(sample_cand_rewards, sample_ref_rewards, sample_cand_probs)
    
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss

        if bleu:
            loss += bleu_loss(batch)
        if bleurt:
            loss += bleurt_loss(bacth)
        if classifier:
            loss += classifier_loss(batch)
                
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
   

    model.eval()
    predictions, references = [], []
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
            )
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        predictions += decoded_preds

        transform = lambda x: [i for i in x]
        references += list(map(list, zip(*map(transform, [batch[f'formal{i}'] for i in range(4)]))))


    results_bleu = metric_bleu.compute(predictions=predictions, references=references)

    print(f"epoch {epoch}, BLEU score: {results_bleu['bleu']:.2f}")

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)

    if results_bleu['bleu']>best_bleu:
        unwrapped_model.save_pretrained('bart-finetuned-fm/best-model', save_function=accelerator.save)
        best_bleu = results_bleu['bleu']
    unwrapped_model.save_pretrained('bart-finetuned-fm', save_function=accelerator.save)


In [136]:
# metric = evaluate.load("bleu")
import evaluate
import numpy as np
from tqdm.auto import tqdm
metric_bleu = evaluate.load('bleu')
# metric_bleurt = evaluate.load('bleurt')
model = AutoModelForSeq2SeqLM.from_pretrained('bart-finetuned-bleurt').to('cuda:0')
predictions, references = [], []
test_dataloader = accelerator.prepare(test_dataloader)
model.eval()
for batch in tqdm(test_dataloader):
    with torch.no_grad():
        generated_tokens = accelerator.unwrap_model(model).generate(
            batch["input_ids"],
            attention_mask=batch["attention_mask"],
            max_length=128,
        )
    labels = batch["labels"]

    # Necessary to pad predictions and labels for being gathered
    generated_tokens = accelerator.pad_across_processes(
        generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
    )
    labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

    predictions_gathered = accelerator.gather(generated_tokens)
    labels_gathered = accelerator.gather(labels)

    decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
    predictions += decoded_preds
    transform = lambda x: [i for i in x]
    references += list(map(list, zip(*map(transform, [batch[f'formal{i}'] for i in range(4)]))))
    
    # metric_bleu.add_batch(predictions=decoded_preds, references=decoded_labels)
    # metric_bleurt.add_batch(predictions=decoded_preds, references=list(map(lambda x: x[0],decoded_labels)))
    inps = clf_tok(decoded_preds , padding=True,truncation=True,max_length=30, return_tensors='pt').to(device)
    # print(inps, inps['input_ids'].shape)
    clf.eval()
    clf.to(device)
    clf_out = clf(**inps)
    clf_probs = torch.nn.functional.softmax(clf_out['logits'], dim=-1)
    clf_score =  clf_probs[:,1] - clf_probs[:,0]
    
    for i in range(1,4):
        bleurt.add_batch(predictions=decoded_preds, references=[i for i in batch[f'formal{i}']])

# results = metric.compute()
results_bleu = metric_bleu.compute(predictions=predictions, references=references)
results_bleurt = bleurt.compute()
# results_bleu = metric_bleu.compute()
# results_bleurt = metric_bleurt.compute()
# print(results_bleu['bleu'], np.mean(results_bleurt['scores']))
print(f"BLEU score: {results_bleu['bleu']:.2f}, BLEURT score: {results_bleurt} ")


  0%|          | 0/45 [00:00<?, ?it/s]

BLEU score: 0.68, BLEURT score: {'scores': [1.0062233209609985, -0.6434895396232605, 0.7893180847167969, 0.1766478568315506, -0.8884022831916809, 0.8143643736839294, 0.43650078773498535, 0.4039737582206726, 0.5815399289131165, -0.03569946065545082, -0.2975192666053772, 0.5529013872146606, -0.1937406212091446, 0.4318162500858307, 0.7718561291694641, 0.6580989956855774, -0.6301640868186951, 0.33151257038116455, 0.7222262620925903, 0.5732143521308899, -0.912496030330658, 0.12779997289180756, 0.06176270171999931, 0.18390901386737823, 0.2898654341697693, 0.020217936486005783, 0.9700319170951843, 0.2980978488922119, -1.1461734771728516, -0.9780104756355286, -0.15536381304264069, 0.3851117193698883, 0.23114842176437378, -0.8426242470741272, 0.7321760058403015, -0.47096002101898193, 0.10994459688663483, 0.47718346118927, 0.7473008036613464, 0.2539657652378082, -1.3021750450134277, 0.02035723254084587, -1.1396780014038086, 0.5988821983337402, -0.6341047883033752, -0.014993656426668167, 0.499327

In [137]:
print(f"BLEU score: {results_bleu['bleu']:.2f}, BLEURT score: {np.mean(results_bleurt['scores'])} ")


BLEU score: 0.68, BLEURT score: 0.07052971926744038 


In [138]:
clf_score.mean()

tensor(0.8376, device='cuda:0', grad_fn=<MeanBackward0>)

In [144]:
data['test'][0]

{'informal': 'Is Any Baby Really A Freak.',
 'formal.ref0': 'Is any baby really a freak.',
 'formal.ref1': 'Is any baby really a freak?',
 'formal.ref2': 'Is Any Baby really strange?',
 'formal.ref3': 'Is anyone really that crazy?'}

In [130]:
sc_preds = predictions

In [132]:
base_preds = predictions

In [139]:
bleurt_preds = predictions

In [162]:
inps = clf_tok('Is any baby really a freak?', padding=True,truncation=True,max_length=30, return_tensors='pt').to(device)
clf.eval()
clf.to(device)
clf_out = clf(**inps)
clf_probs = torch.nn.functional.softmax(clf_out['logits'], dim=-1)
clf_score =  clf_probs[:,1] - clf_probs[:,0]

In [163]:
clf_score

tensor([-0.0808], device='cuda:0', grad_fn=<SubBackward0>)

In [147]:
for f,i, j, k in zip(data['test'],base_preds, bleurt_preds, sc_preds):
    print(f['informal'],'------',i,'-----',j,'--------', k)

Is Any Baby Really A Freak. ------ Is any baby really a freak? ----- Is any baby really a freak? -------- Is any Baby Really Really A Freak?
aspen colorado has he best music festivals, you sit all over the moutians its  on and just hang out ------ Aspenado has the best music festivals, you sit all over the moutians, and just hang out. ----- Aspenado has the best music festivals, you sit all over the moutians, and just hang out. -------- Aspen colorado has his best music festivals, you sit all over the moutians.
You can get almost anything on ebay! ------ You can get almost anything on eBay! ----- You can get almost anything on eBay! -------- You can get almost anything on ebay!
everybody is Dying to get in ------ Everybody is dying to get in. ----- Everybody is dying to get in. -------- Everybody is Dying to get in.
not idiots like 50 cent and his whole Gay unit.those kinds of ppl give hip-hop a bad name. ------ Not idiots like 50 cent and his whole Gay unit. Those kinds of people give

In [39]:
batch['labels']

tensor([[    0,  6025,  1842,   222,    45,   492,   162, 12987,  3985,   293,
          1640,   118,   206,    43,     2,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100],
        [    0,  1116,  9240,  1090,   939,    28,   885,  1488,   179,    24,
          7630,  1506,   183,     6,   127, 23651, 16224,  1488,  1334,    16,
            96,   257, 15144,     2]])

In [40]:
import evaluate

metric = evaluate.load("sacrebleu")

In [41]:
import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [46]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"bart-finetuned-style2",
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=500,
    save_steps=500,
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
    label_smoothing_factor=0.1,
    metric_for_best_model='eval_loss',
    generation_num_beams=5
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [47]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Using cuda_amp half precision backend


In [48]:
trainer.evaluate(max_length=max_length)


***** Running Evaluation *****
  Num examples = 2877
  Batch size = 64


{'eval_loss': 3.764561176300049,
 'eval_bleu': 26.11707161002749,
 'eval_runtime': 73.4402,
 'eval_samples_per_second': 39.175,
 'eval_steps_per_second': 0.613}

In [49]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 52595
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 8220
  Number of trainable parameters = 139420416


Step,Training Loss,Validation Loss,Bleu
500,3.254600,3.729558,25.312853
1000,3.519400,3.664733,26.247859
1500,3.461000,3.629200,26.454366
2000,3.342400,3.635516,26.070765
2500,3.285100,3.585262,26.409308
3000,3.266700,3.596357,26.690683
3500,3.215700,3.606627,26.660400
4000,3.156300,3.574103,26.677689
4500,3.164400,3.584770,26.863489
5000,3.146800,3.577172,26.977730


***** Running Evaluation *****
  Num examples = 2877
  Batch size = 64
Saving model checkpoint to bart-finetuned-style2/checkpoint-500
Configuration saved in bart-finetuned-style2/checkpoint-500/config.json
Model weights saved in bart-finetuned-style2/checkpoint-500/pytorch_model.bin
tokenizer config file saved in bart-finetuned-style2/checkpoint-500/tokenizer_config.json
Special tokens file saved in bart-finetuned-style2/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2877
  Batch size = 64
Saving model checkpoint to bart-finetuned-style2/checkpoint-1000
Configuration saved in bart-finetuned-style2/checkpoint-1000/config.json
Model weights saved in bart-finetuned-style2/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in bart-finetuned-style2/checkpoint-1000/tokenizer_config.json
Special tokens file saved in bart-finetuned-style2/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2877
  Batch size

KeyboardInterrupt: 

In [1]:
trainer.evaluate(max_length=max_length)

NameError: name 'trainer' is not defined